# Sharing Models and Tokenizers on HuggingFace Hub

Learning how to save and share trained models on HuggingFace Hub

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 08/12/2025   | Martin | Created   | Notebook created for model sharing on HF Hub | 

# Content

* [Introduction](#introduction)

In [1]:
%load_ext watermark

# Introduction

- Each model is hosted as a Git repository
- Sharing models on the Hub automatically deploys a hosted Inference API for the model i.e anyone in the community can test and use it

In [ ]:
# Importing using pipeline - Ensure the right pipeline task as stated on model card
from transformers import pipeline

camembert_fill_mask = pipeline("fill-mask", model="camembert-base")
results = camembert_fill_mask("Le camembert est <mask> :)")

In [ ]:
# Importing using specific model card
from transformers import CamembertTokenizer, CamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")

In [ ]:
# Import using Auto* class - Recommended
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")

---

# Sharing Models

- Repository name will be what was specified as the output directory
  - Can be changed with `hub_model_id` argument
- Save frequency based on the `save_strategy` argument
- Final `trainer.push_to_hub()` to save final iteration of model

<u>3 methods to sharing</u>

1. `push_to_hub` API
2. `huggingface_hub` Python library
3. Web interface

## 1. Push to hub

- Requires authentication tokens to indicate user

In [ ]:
# Login to HF hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  "bert-finetuned-mrpc",
  save_strategy="epoch",
  push_to_hub=True # >>: Automatically pushes to a new repository in the Hub
)

In [ ]:
model.push_to_hub("dummy-model")
tokenizer.push_to_hub("dummy-model", organization="huggingface", use_auth_token="<TOKEN>")

## 2. huggingface_hub

Python library that offers tools to interact with the Hub.

- Also requires API token in cache to work

In [ ]:
from huggingface_hub import create_repo

create_repo("dummy-model")

In [ ]:
%watermark